# Complexity: sorting and vectorization

![Creative Commons License](https://i.creativecommons.org/l/by/4.0/88x31.png)  
This work by Jephian Lin is licensed under a [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/).

Note:  Many materials in this notebook file comes from  
the section [Sorting array](https://jakevdp.github.io/PythonDataScienceHandbook/02.08-sorting.html) in  
[*Python Data Science Handbook*](https://jakevdp.github.io/PythonDataScienceHandbook/), O'Reilly Media, 2016 by Jake VanderPlas

## Using NumPy to perform efficient computation
NumPy already implemented various high-performance algorithms.  
Using the idea of vectorization and broadcasing  
allows us to do computation efficiently and effectively.

### Time complexity
A function $f(n)$ is in $O(g(n))$  
if there is a constant $C$ and an integer $N$  
such that $|f(n)|< C|g(n)|$ for any $n\geq N$.

The notation $O(\cdot)$ reads as  
"order" or "big oh".

For example,  
$n^2 + n$ is $O(n^2)$ and in $O(n^3)$,  
$e^n$ is not in $O(n^k)$ for any $k$,  
$2^n$ is in $O(5^n)$.

An algorithm is **polynomial-time** if  
it takes $O(n^k)$ steps to run for some $k$.

An algorithm is **exponential-time** if   
it takes $2^{O(n^k)}$ steps to run for some $k$.

### Sorting
Make the elements of a list  
from small to large.

**Selection sort** keep moving the smallest element  
to the front.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [12]:
arr = np.random.randint(0,11,(10,))
arr

array([ 3,  2, 10,  2,  9,  7,  9,  6, 10,  3])

In [13]:
arr.argmin()

1

In [14]:
def selection_sort(arr):
    new_arr = arr.copy()
    n = len(new_arr)
    for i in range(n):
        min_ind = new_arr[i:].argmin()
        new_arr[i], new_arr[i + min_ind] = new_arr[i + min_ind], new_arr[i]
    return new_arr

In [15]:
selection_sort(arr)

array([ 2,  2,  3,  3,  6,  7,  9,  9, 10, 10])

**Bogosort** is an algorithm that  
keeps shuffling the numbers until  
it is, by luck, from small to large.

In [16]:
def bogosort(arr):
    new_arr = arr.copy()
    again = True
    while again:
        np.random.shuffle(new_arr)
        if np.all(new_arr[:-1] <= new_arr[1:]):
            again = False
    return new_arr

In [17]:
bogosort(arr)

array([ 2,  2,  3,  3,  6,  7,  9,  9, 10, 10])

You may use `%%timeit` to  
test the speed.  

But in general the difference is very obvious.

The time complexity of `selection_sort` is $O(n^2)$,  
while the time complexity of `bogosort` is on average $O(n!)$,  
which is worse than $O(2^n)$.

The best soring algorithms can be done in $O(n\log n)$, and  
they have been implemented in NumPy.

In [18]:
np.sort(arr) ### easy easy

array([ 2,  2,  3,  3,  6,  7,  9,  9, 10, 10])

### Vectorization
The idea of **vectorization** is to   
use efficient matrix/vector algorithm in Numpy (or MATLAB)  
to do the task.  

In [22]:
N = 1000000
a = np.random.randn(N)
b = np.random.randn(N)

In [24]:
%%timeit

c = []
for i in range(N):
    c.append(a[i] + b[i])
np.array(c)

1 loop, best of 3: 605 ms per loop


In [26]:
%%timeit

a + b

100 loops, best of 3: 2.4 ms per loop


Using `for` loops is, in general, slower.  

The computation is not always easy  
to be implemented by built-in functions of NumPy.  

You may still use `np.vectorize` to create one.

For example,  
let $f(x) = \begin{cases} 
1 & \text{if }x\geq 0.5 \\
0 & \text{if }x< 0.5 \\
\end{cases}$.  

Say, you want to apply $f$ to every element  
in an array `a`.  
How to do this?

In [30]:
a = np.random.rand(10)
a

array([0.06708702, 0.90103127, 0.70099116, 0.87192187, 0.0400433 ,
       0.39796714, 0.45084571, 0.31177478, 0.32700703, 0.72718427])

In this special case,  
you may still cheat by the `np.sign` function.  
(And hope there is no entry equal to zero.)

In [31]:
np.sign(a - 0.5)

array([-1.,  1.,  1.,  1., -1., -1., -1., -1., -1.,  1.])

In [32]:
(np.sign(a - 0.5) + 1)/2

array([0., 1., 1., 1., 0., 0., 0., 0., 0., 1.])

Here is a better way.

In [33]:
def f(x):
    if x >= 0.5:
        return 1
    else:
        return 0
    
vec_f = np.vectorize(f)

In [34]:
vec_f(a)

array([0, 1, 1, 1, 0, 0, 0, 0, 0, 1])

### Boardcasting
NumPy allow binary operations to be applied  
to two arrays of different sizes.  
This behavior is called **broadcasting**.

In [9]:
a = np.arange(3).reshape(3,1) ### a column
a

array([[0],
       [1],
       [2]])

In [10]:
b = np.arange(3) ### a one-dimensional sequence
b

array([0, 1, 2])

In [11]:
a + b

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

Before understanding the broadcasting rules.  
Let's recall some terms.

The **shape** is a tuple indicating the "height" and the "width".  
The **number of dimensions** (`ndim`) is the number of axes.

In [12]:
print(a)
print(a.shape)
print(a.ndim)

[[0]
 [1]
 [2]]
(3, 1)
2


In [13]:
print(b)
print(b.shape)
print(b.ndim)

[0 1 2]
(3,)
1


The broadcasting rules:
1. If `a.ndim > b.ndim`, pad some `1`'s on the left of `a.shape`.
2. When `a.ndim == b.ndim`, if `a.shape[k], b.shape[k] == 1, d`  
then duplicate `a` by `d` times so that `a.shape[k] == b.shape[k]`.
3. When `a.ndim == b.ndim`, if `a.shape[k] != b.shape[k]` for some `k`  
and none of them is `1`, then raise an error.

In [2]:
def print_ab(a, b):
    print("a =")
    print(a)
    print()
    print(a.ndim, a.shape)
    print("---")
    print("b =")
    print(b)
    print()
    print(b.ndim, b.shape)

In [3]:
a = np.arange(3).reshape(3,1)
b = np.arange(3)

print_ab(a, b)

a =
[[0]
 [1]
 [2]]

2 (3, 1)
---
b =
[0 1 2]

1 (3,)


In [4]:
b = b[np.newaxis, :]

print_ab(a, b)

a =
[[0]
 [1]
 [2]]

2 (3, 1)
---
b =
[[0 1 2]]

2 (1, 3)


In [5]:
a = a * np.ones(3)
b = b * np.ones(3)[:, np.newaxis]

print_ab(a, b)

a =
[[0. 0. 0.]
 [1. 1. 1.]
 [2. 2. 2.]]

2 (3, 3)
---
b =
[[0. 1. 2.]
 [0. 1. 2.]
 [0. 1. 2.]]

2 (3, 3)


Now add `a` and `b`.

In [28]:
a + b

array([[2., 3., 4.],
       [3., 4., 5.],
       [4., 5., 6.]])

##### Exercise
Implement the selection sort  
by the function `selection_sort_rev`  
so that the output is from the largest to the smallest.

In [2]:
### your answer here


##### Exercise
Obtain `arr` by the following.
```Python
arr = np.random.randint(0, 11, (5,7))
```
Apply `np.sort` to `arr` and  
guess the meaning of the output.

In [2]:
### your answer here


##### Exercise
Obtain `arr` by the following.
```Python
arr = np.random.randint(0, 11, (5,7))
```
Use `np.sort` to sort each column of `arr`.

In [2]:
### your answer here


##### Exercise
Obtain `arr` by the following.
```Python
arr = np.random.randint(0, 11, (15,))
```
Apply `np.argsort` to `arr` and  
guess the meaning of the output.

In [2]:
### your answer here


##### Exercise
Obtain `arr` by the following.
```Python
arr = np.random.randint(0, 11, (15,))
```
Run `np.partition(arr, 3)` and  
guess the meaning of the output.

In [2]:
### your answer here


##### Exercise
Obtain `arr` by the following.
```Python
arr = np.random.randint(0, 11, (15,))
```
Run `np.argpartition(arr, 3)` and  
guess the meaning of the output.

In [2]:
### your answer here


##### Exercise
Create a function `f(x)` that returns   
`0` if `x < 0.5`,  
`1` if `0.5 <= x <= 1.5`, and  
`2` if `x > 1.5`.  

Then create a vectorized function from `f`.

In [2]:
### your answer here


##### Exercise
Let `a = np.arange(3)`.  
Print the shapes of  
`a[:, np.newaxis]` and `a[np.newaxis, :]`.

In [2]:
### your answer here


##### Exercise
Let `a = np.random.randint(0, 10, (5,7))`  
and `mean = a.mean(axis=1)`.  

Adjust the shape of `a` or `mean` if necessary,  
then use `a - mean` to normalize each row,  
so that each row has mean equal to zero.

In [2]:
### your answer here


##### Exercise
Let `X = np.random.randn(5,2)`.  

Thinking `X` as `5` samples of points in $\mathbb{R}^2$,  
what is the meaning of  
`X[:, np.newaxis, :] - X[np.newaxis, :, :]`?

In [2]:
### your answer here
